## MuseScore Scraper

Tool: https://github.com/frankye8998/MusicalMusic  

[Reference 2](https://github.com/codeandproduce/music_research_dataset_midi/blob/9a5afe86f1f2d3fdbfa24de99122140e7a7cd60c/MuseScore/MuseScore%20Scraping%20Main.ipynb)

In [1]:
from lxml import html
import urllib
import requests
import shutil

In [2]:
from random import randint
from time import sleep

In [3]:
import json
from pathlib import Path
from tqdm import tqdm
import time

In [4]:
# links = []
# url = "https://musescore.com/hub/piano/solo-piano?sort=view_count"
# # url = "https://musescore.com/hub/piano/piano-duet?sort=view_count"
# page = requests.get(url)
# tree = html.fromstring(page.content)
# print(tree.xpath("//title/text()"))

In [5]:
# articles = tree.xpath("//article[@role='article']")

In [6]:
def get_data(a):
    score = a.find("h2//a[@rel='bookmark']")
    link = score.attrib['href']
    title = score.text_content().strip()
    author = a.find("div[@class='user']//a").text_content()
    metadata = a.find("div[@class='meta']").text_content().split('•\n')
    parts, pages, duration, time, views = [m.strip() for m in metadata]
    return {
        'link': "https://musescore.com" + link,
        'score_id': Path(link).name,
        'title': title,
        'author': author,
        'parts': parts,
        'pages': pages,
        'duration': duration,
        'views': views
    }

## But that's actually just page one and there are pages 2,3,4,5... 100!
### So let's make a more general format:

In [8]:
# url_default = "https://musescore.com/hub/video_games/movie?sort=view_count&page=" # + the page number
url_default = 'https://musescore.com/hub/piano/easy-piano?sort=view_count&page='
# url_default = "https://musescore.com/hub/piano/solo-piano?sort=view_count&page=" # + the page number
# url_default = "https://musescore.com/hub/piano/voice-piano?sort=view_count&page=" # + the page number
# url_default = "https://musescore.com/hub/piano?sort=view_count&page=" # + the page number

In [9]:
json_file = Path('musescore_piano_easy.json')

In [10]:
if json_file.exists():
    with open(json_file, 'r') as fp:
        links = json.load(fp)
        

In [11]:
if not json_file.exists():
    links = []
    for page in tqdm(range(1,100), total=100):
        page = requests.get(url_default+str(page)) # https://musescore.com/hub/piano?page=1,2,3,4,5,...,10
        tree = html.fromstring(page.content)

        articles = tree.xpath("//article[@role='article']")

        links.extend([get_data(a) for a in articles])

#         sleep(randint(1,4))
    with open(json_file, 'w') as fp:
        json.dump(links, fp)
print(links[:100])

 99%|█████████▉| 99/100 [01:07<00:00,  1.80it/s]

[{'link': 'https://musescore.com/user/158751/scores/2163051', 'score_id': '2163051', 'title': 'Game of Thrones, Easy piano', 'author': 'lucky37', 'parts': '1 part', 'pages': '3 pages', 'duration': '01:39', 'views': '448,392 views'}, {'link': 'https://musescore.com/user/2466621/scores/2100881', 'score_id': '2100881', 'title': 'Requiem for a Dream (Easy)', 'author': 'Torby Brand', 'parts': '1 part', 'pages': '2 pages', 'duration': '01:48', 'views': '137,403 views'}, {'link': 'https://musescore.com/user/13543696/scores/4836661', 'score_id': '4836661', 'title': 'Someone Like You (easy piano)', 'author': 'matikavi11869', 'parts': '1 part', 'pages': '6 pages', 'duration': '04:48', 'views': '127,567 views'}, {'link': 'https://musescore.com/user/2500616/scores/2311476', 'score_id': '2311476', 'title': 'Star Wars Theme - easy piano', 'author': 'Erin Keely Whitworth Machado', 'parts': '1 part', 'pages': '1 page', 'duration': '00:58', 'views': '126,604 views'}, {'link': 'https://musescore.com/use

# Scrape links found

### Musical Music Lib

In [12]:

class MuseScoreException(Exception):
    pass

class InvalidFileExtension(MuseScoreException):
    pass

class InvalidScoreID(MuseScoreException):
    pass

class InvalidCredentials(MuseScoreException):
    pass

class InvalidSearchSort(MuseScoreException):
    pass

In [13]:
import urllib
import urllib.request


import bs4
import requests

class MusicalMusic:
    """Musescore actions requiring an account."""

    def __init__(self, username, password):
        self.username = username
        url = "https://musescore.com/user/login"
        r = requests.get(url)
        soup = bs4.BeautifulSoup(r.text, "html.parser")
        csrf = soup.find("meta", {"name": "csrf-token"})["content"]
        url = "https://musescore.com/user/auth/login/process"
        cookies = {
            "mu_browser_uni": r.cookies['mu_browser_uni'],
            "_csrf": r.cookies["_csrf"]
        }
        data = {
          "username": username,
          "password": password,
          "_csrf": csrf,
          "op": "Log in"
        }
        try:
            mu_user = requests.post(url,
                                    data=data,
                                    cookies=cookies,
                                    allow_redirects=False,
                                    ).cookies["mu_user_new"]
        except KeyError as e:
            raise InvalidCredentials(
                "Please check your username and password!") from e

        mu_browser_uni = r.cookies['mu_browser_uni']

        self.mu_browser_uni = mu_browser_uni
        self.mu_user = mu_user

    def retrieve(self, id, format="pdf"):
        """Retrieves Musescore data in bytes"""

        if format not in ["mp3", "pdf", "mid", "mxl", "mscz"]:
            raise InvalidFileExtension("Must be mp3, pdf, mid, mxl, or mscz.")
        newlink = f"https://musescore.com/score/{id}/download/{format}"
        cookies = {"mu_browser_uni": self.mu_browser_uni,
                   "mu_user_new": self.mu_user}
        bytes = requests.get(newlink, cookies=cookies, verify=False)
        if bytes.status_code != 200:
            raise InvalidScoreID(str(bytes.status_code))
        return bytes.content

    def download(self, id, filename, format="mp3", proxy=None):
        if format not in ["mp3", "pdf", "mid", "mxl", "mscz"]:
            raise InvalidFileExtension("Must be mp3, pdf, mid, mxl, or mscz.")
        newlink = f"https://musescore.com/score/{id}/download/{format}"
        if proxy:
            proxy_support = urllib.request.ProxyHandler({'https' : proxy})
            opener = urllib.request.build_opener(proxy_support)
        else: opener = urllib.request.build_opener()
            
        cookieString = f"mu_browser_uni={self.mu_browser_uni};" \
                        f"mu_user_new={self.mu_user}"
        opener.addheaders = [("cookie", cookieString)]
        urllib.request.install_opener(opener)
        try:
            urllib.request.urlretrieve(newlink, filename)
        except urllib.error.HTTPError as e:
            raise e


### Trying proxy

https://codelike.pro/create-a-crawler-with-rotating-ip-proxy-in-python/

In [14]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random

ua = UserAgent() # From here we generate a random user agent
proxies = [] # Will contain proxies [ip, port]

In [15]:
# Main function
  # Retrieve latest proxies
proxies_req = Request('https://www.sslproxies.org/')
proxies_req.add_header('User-Agent', ua.random)
proxies_doc = urlopen(proxies_req).read().decode('utf8')

soup = BeautifulSoup(proxies_doc, 'html.parser')
proxies_table = soup.find(id='proxylisttable')

# Save proxies in the array
for row in proxies_table.tbody.find_all('tr'):
    proxies.append({
      'ip':   row.find_all('td')[0].string,
      'port': row.find_all('td')[1].string
    })

### More proxies

https://github.com/constverum/ProxyBroker

In [16]:
import asyncio
from proxybroker import Broker

more_proxies = []

async def show(proxy_queue):
    while True:
        proxy = await proxy_queue.get()
        if proxy is None: break
        print('Found proxy: %s' % proxy)
        more_proxies.append({ 'ip': proxy.host, 'port': proxy.port })

proxy_queue = asyncio.Queue()
broker = Broker(proxy_queue)
tasks = await asyncio.gather(
    broker.find(types=['HTTPS'], limit=200),
    show(proxy_queue))

Found proxy: <Proxy FR 0.15s [HTTPS] 54.39.97.250:3128>
Found proxy: <Proxy EC 0.28s [HTTPS] 186.46.220.117:80>
Found proxy: <Proxy MX 0.32s [HTTPS] 187.216.93.20:53281>
Found proxy: <Proxy US 0.33s [HTTPS] 173.249.0.209:3128>
Found proxy: <Proxy DE 0.35s [HTTPS] 94.130.126.94:8008>
Found proxy: <Proxy JP 0.36s [HTTPS] 160.16.52.185:3128>
Found proxy: <Proxy SK 0.38s [HTTPS] 185.152.112.18:37900>
Found proxy: <Proxy RU 0.36s [HTTPS] 94.242.59.245:1448>
Found proxy: <Proxy PL 0.41s [HTTPS] 78.11.118.157:3128>
Found proxy: <Proxy RU 0.38s [HTTPS] 94.242.59.135:1448>
Found proxy: <Proxy RU 0.39s [HTTPS] 82.138.23.141:3128>
Found proxy: <Proxy BR 0.40s [HTTPS] 200.255.122.170:8080>
Found proxy: <Proxy SE 0.43s [HTTPS] 46.246.38.90:3128>
Found proxy: <Proxy RU 0.44s [HTTPS] 185.41.112.29:57190>
Found proxy: <Proxy PL 0.45s [HTTPS] 91.218.63.196:53281>
Found proxy: <Proxy -- 0.45s [HTTPS] 85.209.163.68:8080>
Found proxy: <Proxy US 0.48s [HTTPS] 68.183.180.184:8080>
Found proxy: <Proxy UA 0.4

Found proxy: <Proxy BR 0.61s [HTTPS] 181.191.180.110:8080>
Found proxy: <Proxy NL 0.28s [HTTPS] 176.56.236.158:3128>
Found proxy: <Proxy ES 2.39s [HTTPS] 178.60.28.98:9999>
Found proxy: <Proxy MX 2.41s [HTTPS] 200.66.94.147:8080>
Found proxy: <Proxy PL 0.27s [HTTPS] 145.239.87.173:3128>
Found proxy: <Proxy ID 0.48s [HTTPS] 139.255.25.83:3128>
Found proxy: <Proxy -- 0.39s [HTTPS] 45.71.38.77:999>
Found proxy: <Proxy CZ 2.50s [HTTPS] 85.207.44.10:53038>
Found proxy: <Proxy ID 0.66s [HTTPS] 103.228.117.244:8080>
Found proxy: <Proxy BR 0.69s [HTTPS] 138.219.223.166:3128>
Found proxy: <Proxy IN 0.98s [HTTPS] 45.250.226.10:8080>
Found proxy: <Proxy RU 2.15s [HTTPS] 176.196.84.138:51336>
Found proxy: <Proxy TH 0.59s [HTTPS] 202.28.17.5:8080>
Found proxy: <Proxy TH 2.58s [HTTPS] 180.180.156.35:37463>
Found proxy: <Proxy RU 1.72s [HTTPS] 185.5.19.234:52975>
Found proxy: <Proxy ID 1.45s [HTTPS] 103.76.15.138:47847>
Found proxy: <Proxy BY 0.38s [HTTPS] 86.57.219.179:23500>
Found proxy: <Proxy FR 

### Random proxy

In [17]:
all_proxies = proxies + more_proxies
# all_proxies = more_proxies
# Retrieve a random index proxy (we need the index to delete it if not working)
def random_proxy():
    return random.randint(0, len(all_proxies) - 1)

proxy = None
# # Choose a random proxy
# proxy_index = random_proxy()
# proxy = all_proxies[proxy_index]
# proxy_url = proxy['ip'] + ':' + str(proxy['port']); proxy_url

## Actual scraping

In [18]:
invalid_ids = []

In [19]:
deleted_proxies = []

In [20]:
with open('accounts.json', 'r') as fp:
    accounts = json.load(fp)

instances = [MusicalMusic(username, password) for (username, password) in accounts]
instance = None

In [37]:
for idx,link in enumerate(tqdm(links, total=len(links))):
    score_id = link['score_id']
    out_file = f"data/{score_id}.mxl"
    if Path(out_file).exists() or score_id in invalid_ids: continue
    # Every 10 requests, generate a new proxy
    if idx % 10 == 0 or instance is None:
        instance_index = random.randint(0, len(instances) - 1)
        instance = instances[instance_index]
    try:
        print('Downloading score id:', score_id)
        instance.download(score_id, out_file, format='mxl')
    except Exception as e:
        print('Could not download id:', score_id)
        instance = None
    sleep(randint(1,2))
    







  0%|          | 0/762 [00:00<?, ?it/s]







 73%|███████▎  | 553/762 [00:02<00:00, 254.34it/s]







 73%|███████▎  | 553/762 [00:13<00:00, 254.34it/s]





 73%|███████▎  | 558/762 [00:16<02:50,  1.19it/s] 







 73%|███████▎  | 559/762 [00:18<04:51,  1.43s/it]







 73%|███████▎  | 560/762 [00:20<05:01,  1.49s/it]

...







 99%|█████████▉| 758/762 [08:29<00:11,  2.81s/it]







100%|█████████▉| 759/762 [08:31<00:07,  2.60s/it]







100%|█████████▉| 760/762 [08:34<00:05,  2.60s/it]







100%|█████████▉| 761/762 [08:37<00:02,  2.78s/it]







100%|██████████| 762/762 [08:39<00:00,  2.43s/it]







In [21]:
len(all_proxies)

300

In [35]:

# del all_proxies[proxy_index]
# proxy = None

In [36]:
for idx,link in enumerate(tqdm(links, total=len(links))):
    score_id = link['score_id']
    out_file = f"data/{score_id}.mxl"
    if Path(out_file).exists() or score_id in invalid_ids: continue
    # Every 10 requests, generate a new proxy
    if idx % 10 == 0 or proxy is None or instance is None:
        proxy_index = random.randint(0, len(all_proxies) - 1)
        proxy = all_proxies[proxy_index]
        proxy_url = proxy['ip'] + ':' + str(proxy['port']); proxy_url
        instance_index = random.randint(0, len(instances) - 1)
        instance = instances[instance_index]
    try:
        print('Downloading score id:', score_id)
        
        start = time.time()
        instance.download(score_id, out_file, format='mxl', proxy=proxy_url)
        end = time.time()
        if (end - start) > 15: raise Exception('Took too long to download')
    except Exception as e:
        print('Could not download id:', score_id)
        print('Error:', e)
        invalid_ids.append(score_id)
        deleted_proxies.append(all_proxies[proxy_index])
        del all_proxies[proxy_index]
        print('Proxy ' + proxy['ip'] + ':' + str(proxy['port']) + ' deleted.')
        print(instance.username)
        proxy = None
        instance = None
    






  0%|          | 0/762 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Second run

In [23]:
# Main function
  # Retrieve latest proxies
proxies_req = Request('https://www.sslproxies.org/')
proxies_req.add_header('User-Agent', ua.random)
proxies_doc = urlopen(proxies_req).read().decode('utf8')

soup = BeautifulSoup(proxies_doc, 'html.parser')
proxies_table = soup.find(id='proxylisttable')

# Save proxies in the array
for row in proxies_table.tbody.find_all('tr'):
    proxies.append({
      'ip':   row.find_all('td')[0].string,
      'port': row.find_all('td')[1].string
    })

HTTPError: HTTP Error 403: Forbidden

In [ ]:
import asyncio
from proxybroker import Broker

more_proxies = []

async def show(proxy_queue):
    while True:
        proxy = await proxy_queue.get()
        if proxy is None: break
        print('Found proxy: %s' % proxy)
        more_proxies.append({ 'ip': proxy.host, 'port': proxy.port })

proxy_queue = asyncio.Queue()
broker = Broker(proxy_queue)
tasks = await asyncio.gather(
    broker.find(types=['HTTP', 'HTTPS'], limit=400),
    show(proxy_queue))

In [ ]:
all_proxies = proxies + more_proxies
# Retrieve a random index proxy (we need the index to delete it if not working)
def random_proxy():
    return random.randint(0, len(all_proxies) - 1)

proxy = None
# # Choose a random proxy
# proxy_index = random_proxy()
# proxy = all_proxies[proxy_index]
# proxy_url = proxy['ip'] + ':' + str(proxy['port']); proxy_url

In [ ]:
json_file = Path('musescore_movie.json')

In [ ]:
if json_file.exists():
    with open(json_file, 'r') as fp:
        links = json.load(fp)
        

In [ ]:
for idx,link in enumerate(tqdm(links, total=len(links))):
    score_id = link['score_id']
    out_file = f"data/{score_id}.mxl"
    if Path(out_file).exists() or score_id in invalid_ids: continue
    # Every 10 requests, generate a new proxy
    if idx % 10 == 0 or proxy is None or instance is None:
        proxy_index = random.randint(0, len(all_proxies) - 1)
        proxy = all_proxies[proxy_index]
        proxy_url = proxy['ip'] + ':' + str(proxy['port']); proxy_url
        instance_index = random.randint(0, len(instances) - 1)
        instance = instances[instance_index]
    try:
        print('Downloading score id:', score_id)
        instance.download(score_id, out_file, format='mxl', proxy=proxy_url)
    except Exception as e:
        print('Could not download id:', score_id)
        print('Error:', e)
        invalid_ids.append(score_id)
        deleted_proxies.append(all_proxies[proxy_index])
        del all_proxies[proxy_index]
        print('Proxy ' + proxy['ip'] + ':' + str(proxy['port']) + ' deleted.')
        print(instance.username)
        proxy = None
        instance = None
    sleep(randint(1,2))
    